In [7]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point
import plotly_express as px

### Municipal votes count for a specific vereador of Porto Alegre

In [8]:
df = pd.read_csv(r"CSV/votacao_secao_2020_RS.csv", encoding="Latin 1", sep=";")
df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_MUNICIPIO',
       'NM_MUNICIPIO', 'NR_ZONA', 'NR_SECAO', 'CD_CARGO', 'DS_CARGO',
       'NR_VOTAVEL', 'NM_VOTAVEL', 'QT_VOTOS', 'NR_LOCAL_VOTACAO'],
      dtype='object')

#### Import geocoded places

In [9]:
geocoded_df = pd.read_csv("CSV/geocoded_voting_places_001.csv")
df_lat_long = df.merge(geocoded_df, on = ['CD_MUNICIPIO', 'NR_ZONA', 'NR_LOCAL_VOTACAO'], how = 'left')
df_lat_long.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,...,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_LOCAL_VOTACAO,NR_LATITUDE,NR_LONGITUDE
0,21/12/2020,16:55:14,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,51,150,11,PREFEITO,13,ARY JOSÉ VANAZZI,98,1031,-29.772053,-51.147099
1,21/12/2020,16:55:14,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,51,150,11,PREFEITO,23,RONALDO TEIXEIRA DA SILVA,58,1031,-29.772053,-51.147099
2,21/12/2020,16:55:14,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,51,150,11,PREFEITO,25,HELIOMAR ATHAYDES FRANCO,81,1031,-29.772053,-51.147099
3,21/12/2020,16:55:14,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,51,150,11,PREFEITO,50,CÉLIO JULIANO BARROSO TRINDADE,3,1031,-29.772053,-51.147099
4,21/12/2020,16:55:14,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,...,51,150,11,PREFEITO,95,VOTO BRANCO,8,1031,-29.772053,-51.147099


#### Import nomes de locais

In [83]:
nomeslocais = pd.read_csv("CSV/NomesLocais.csv")
nomeslocais.head()

,Unnamed: 0,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,NM_LOCAL_VOTACAO,DS_LOCAL_VOTACAO_ENDERECO
0,0,SELVÍRIA,9,157,1015,CENTRO ADMINISTRATIVO FABIANO MELO (ANTIGA FAC...,"AV. GOIÁS, 556"
1,1,PELOTAS,60,500,2526,UNIÃO GAÚCHA J. SIMÕES LOPES NETO,"AV. ENGENHEIRO ILDEFONSO SIMÕES LOPES, 531"
2,2,BELO HORIZONTE,35,221,1180,UTRAMIG,"AVENIDA AFONSO PENA, 3400"
3,3,POTÉ,270,92,1023,ESCOLA ESTADUAL GUIMARAES ROSA,"RUA ELIEZER PENA, 177"
4,4,SÃO JOÃO DE MERITI,187,300,1139,COLÉGIO ESTADUAL RUBENS FARRULA,"AV. PLACIDO FIGUEIREDO JUNIOR, S/N"


### Merge geocoded and nome locais

In [84]:
df_votos_vereador_nmlocal = pd.merge(df_lat_long, nomeslocais, on = ['NM_MUNICIPIO','NR_ZONA','NR_SECAO'])
df_votos_vereador_nmlocal.shape

(1951552, 29)

#### Check for Nome do Vereador

In [92]:
df_votos_vereador = df_votos_vereador_nmlocal[df_votos_vereador_nmlocal['NM_VOTAVEL']=='LUCAS SALOMÓN DA SILVA FUHR'][['NM_MUNICIPIO','NM_VOTAVEL','NM_LOCAL_VOTACAO', 'NR_ZONA','QT_VOTOS']]

#### Dataquality check

In [93]:
df_votos_vereador['QT_VOTOS'].sum()

717

#### Export

In [97]:
df_votos_vereador_g = df_votos_vereador[['NR_ZONA','NM_LOCAL_VOTACAO','QT_VOTOS']].groupby(by=['NR_ZONA','NM_LOCAL_VOTACAO']).sum('QT_VOTOS')
df_votos_vereador_g.to_excel('votos_fuhr_2020.xlsx', merge_cells=False)